In [8]:
import pandas as pd
import sys, os

sys.path.append(os.path.abspath(".."))

# Load data
financials = pd.read_csv("../data/cleaned/financial_statements.csv")

# Imports
from engines.composite_risk_engine import composite_risk_engine
from engines.ratio_engine_core import ratio_engine
from engines.trend_engine import trend_engine
from engines.cash_flow_engine import cash_flow_engine
from engines.anomaly_efficiency_engine import anomaly_efficiency_engine
from engines.solvency_engine import solvency_engine


In [9]:
ratio_results = ratio_engine(financials)

ratios_df = pd.DataFrame(ratio_results)
ratios_df.sort_values(["Company", "Year"], inplace=True)
ratios_df.reset_index(drop=True, inplace=True)
trend_results = trend_engine(ratios_df)

cash_flow_results = cash_flow_engine(financials)
anomaly_results = anomaly_efficiency_engine(ratios_df)
solvency_results = solvency_engine(ratios_df)

In [10]:
composite_results = composite_risk_engine(
    ratio_results,
    trend_results,
    cash_flow_results,
    anomaly_results,
    solvency_results
)

composite_results


[{'engine': 'composite_risk_engine',
  'Company': 'Banyan Retail Co',
  'Year': 2020,
  'risk_score': 0.0,
  'risk_tier': 'Low Risk',
  'components': {'ratios': 0,
   'trends': 0.0,
   'cashflow': 0,
   'anomalies': 0,
   'solvency': 0}},
 {'engine': 'composite_risk_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': 2023,
  'risk_score': 20.0,
  'risk_tier': 'Low Risk',
  'components': {'ratios': 0,
   'trends': 0.0,
   'cashflow': 0,
   'anomalies': 1,
   'solvency': 0}},
 {'engine': 'composite_risk_engine',
  'Company': 'Banyan Retail Co',
  'Year': 2023,
  'risk_score': 20.0,
  'risk_tier': 'Low Risk',
  'components': {'ratios': 0,
   'trends': 0.0,
   'cashflow': 0,
   'anomalies': 1,
   'solvency': 0}},
 {'engine': 'composite_risk_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': 2021,
  'risk_score': 20.0,
  'risk_tier': 'Low Risk',
  'components': {'ratios': 0,
   'trends': 0.0,
   'cashflow': 0,
   'anomalies': 1,
   'solvency': 0}},
 {'engine': 'composite_risk_e

In [7]:
pd.DataFrame(composite_results)

,engine,Company,Year,risk_score,risk_tier,components
0,composite_risk_engine,Banyan Retail Co,2020,0.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
1,composite_risk_engine,Acme Manufacturing Ltd,2023,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
2,composite_risk_engine,Banyan Retail Co,2023,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
3,composite_risk_engine,Acme Manufacturing Ltd,2021,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
4,composite_risk_engine,Coastal Tech Ltd,2021,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
5,composite_risk_engine,Acme Manufacturing Ltd,2020,10.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
6,composite_risk_engine,Emerald Energy Inc,2022,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
7,composite_risk_engine,Banyan Retail Co,2021,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
8,composite_risk_engine,Coastal Tech Ltd,2023,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."
9,composite_risk_engine,Emerald Energy Inc,2023,20.0,Low Risk,"{'ratios': 0, 'trends': 0.0, 'cashflow': 0, 'a..."


In [13]:
df = pd.DataFrame(composite_results)

df["risk_score"].describe()


count    16.000000
mean     16.250000
std       7.187953
min       0.000000
25%      17.500000
50%      20.000000
75%      20.000000
max      20.000000
Name: risk_score, dtype: float64

In [14]:
df.groupby("risk_tier").size()


risk_tier
Low Risk    16
dtype: int64